<a href="https://colab.research.google.com/github/NishiTa-ai/composio/blob/master/smash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Install necessary libraries
!pip install transformers
!pip install pdfplumber
!pip install pytesseract
!apt-get update
!apt-get install -y tesseract-ocr
!pip install Pillow


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [17]:
import pdfplumber
import pytesseract
from PIL import Image
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


In [18]:
import os

# Define paths to your data
claims_forms_path = "/path/to/claims/forms/"
insurance_policies_path = "/path/to/insurance/policies/"
historical_claims_data_path = "/path/to/historical/claims/data/"

# Example function to load and preprocess data
def load_data(path):
    data = []
    if os.path.exists(path):  # Ensure the path exists
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                data.append(f.read())
    else:
        print(f"Path does not exist: {path}")
    return data

claims_forms = load_data(claims_forms_path)
insurance_policies = load_data(insurance_policies_path)
historical_claims_data = load_data(historical_claims_data_path)


Path does not exist: /path/to/claims/forms/
Path does not exist: /path/to/insurance/policies/
Path does not exist: /path/to/historical/claims/data/


In [19]:
def extract_text_from_pdf(file_path):
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
                else:
                    print(f"Empty page found in PDF: {file_path}")
    except Exception as e:
        print(f"Error extracting text from PDF {file_path}: {e}")
    return text

def extract_text_from_image(image_path):
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
    except Exception as e:
        print(f"Error extracting text from image {image_path}: {e}")
        text = ""
    return text


In [20]:
# Load a pre-trained language model for text extraction
try:
    tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
    model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
except Exception as e:
    print(f"Error loading model: {e}")

# Text extraction function using LLM
def extract_data_using_llm(text):
    try:
        inputs = tokenizer(text, return_tensors="pt")
        summary_ids = model.generate(inputs["input_ids"])
        extracted_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Error during data extraction using LLM: {e}")
        extracted_text = ""
    return extracted_text


In [21]:
def verify_policy(extracted_claim_data, insurance_policies):
    try:
        for policy in insurance_policies:
            if extracted_claim_data in policy:
                return True
    except Exception as e:
        print(f"Error in policy verification: {e}")
    return False

# Initialize a text classification pipeline
try:
    nlp_pipeline = pipeline("text-classification", model="distilbert-base-uncased")
except Exception as e:
    print(f"Error loading NLP pipeline: {e}")

def initial_claim_decision(claim_data):
    try:
        result = nlp_pipeline(claim_data)
        decision = result[0]["label"]  # Example labels: 'approve', 'deny', 'request more info'
    except Exception as e:
        print(f"Error in initial claim decision: {e}")
        decision = "error"
    return decision


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
